In [3]:
import torch
import math

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [5]:
vocab_size = 100
n_embed = 64
max_tokens = 10


In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads = 8, device = 'gpu'):
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self.device = device

        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential([
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(device=device),
            nn.Linear(hidden_size, hidden_size, device=device),
        ], device=device
        )
        self.add_norm = nn.Sequential([
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU(device=device)
        ], device=device
        )
        self.add_norm2 = nn.Sequential([
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU(device=device)
        ],device=device
        )

    def forward(self, x):
        res, _ = self.attention(x, x, x) 
        x = self.add_norm1(res + x)
        res = self.ffwd(x)
        x = self.add_norm2(res + x)
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, device='gpu'):
        super(Decoder, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._device = device

        self.masked_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.enc_dec_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential([
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(device=device),
            nn.Linear(hidden_size, hidden_size, device=device)
        ], device=device
        )
        self.add_norm1 = nn.Sequential([
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU(device=device)
        ], device=device
        )
        self.add_norm2 = nn.Sequential([
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU(device=device)
        ], device=device
        )
        self.add_norm3 = nn.Sequential([
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU(device=device)
        ], device=device
        )

    def forward(self, x, hidden_state):
        res, _ = self.masked_attention.forward(x, x, x, 
                                               attn_mask=torch.tril(
                                                   torch.eye(self._hidden_size, 
                                                             self._hidden_size, 
                                                             dtype=torch.bool,
                                                             device=self._device))) #! test mask
        x = self.add_norm1(res + x)
        res, _ = self.enc_dec_attention.forward(x, hidden_state, hidden_state) #! check if its right
        x = self.add_norm2(res + x)
        res = self.ffwd(x)
        x = self.add_norm3(res + x)
        return x

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads = 8, n_encoders=6, device='gpu'):
        super(EncoderBlock, self).__init__()
        
        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_encoders = n_encoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)

        self._encoders = [Encoder(input_size, hidden_size, output_size, dropout, num_heads, device) for _ in range(n_encoders)] 

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        
        for encoder in self._encoders:
            x = encoder(x)
            
        return x

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, n_decoders=6, device='gpu'):
        super(DecoderBlock, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_decoders = n_decoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)
        self.linear = nn.Linear(hidden_size, output_size, device=device)
        self.softmax = nn.Softmax(dim=1, device=device)

        self._decoders = [Decoder(
            input_size, hidden_size, output_size, dropout, num_heads, device=device) for _ in range(n_decoders)]

    def forward(self, x, hidden_state):
        x = self.embedding(x)
        x = self.pos_encoder(x)

        for decoder in self._decoders:
            x = decoder(x, hidden_state)
        
        x = self.linear(x)
        x = self.softmax(x)

        return x